In [51]:
%run DataProcessing.ipynb
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)
from IPython.display import display,HTML

In [74]:
property = pd.read_excel('../01- Compensation Calculation/Working/Data/Property_Cohost.xlsx')
data = import_data()
idx = (data['checkin_date']>=pd.to_datetime('2024-01-01')) & (data['checkin_date']<pd.to_datetime('2025-09-30'))
guest2425 = data.loc[idx].copy()
guest2425['Count'] = np.where((guest2425['checkin_date']<pd.to_datetime('2025-03-31'))&(guest2425['nights']>14),14,
                                     guest2425['nights'])* guest2425['guests']
supplies = pd.read_excel('../Data/Supplies_allunits_2425.xlsx')
supplies['yearmonth']=supplies['Transaction date'].astype(str).str[:7]

In [96]:
sum_supplies = supplies.groupby('yearmonth').agg({'Amount':'sum'}).reset_index()
nights_property= guest2425.groupby(['month','Listing']).agg({'Count':'sum'}).reset_index().\
    merge(property[['Listing','City',"Region"]], on='Listing', how='left')
idx = nights_property['Region'].isin(['Seattle','Eastside'])
nights_property = nights_property.loc[idx].copy()
idx_710 = (nights_property['Listing'].isin(["Seattle 710","Seattle 710 ADU"])) & \
           (nights_property['month'].astype(str).str[:4]=='2024')
nights_property = nights_property.loc[~idx_710].copy()
idx_ownerpay= nights_property['Listing'].isin(["Bellevue 10409","Bellevue 242",\
        'Bellevue 701','Bellevue 514','Mercer 3627 Main','Mercer 3627',"Mercer 3627 ADU",\
        'Seattle 1623','Seattle 3617',"Elektra 1305","Seattle 9021","Bellevue 16237",\
        "Bellevue 14507","Redmond 18641","Beachwood 5"])
nights_property = nights_property.loc[~idx_ownerpay].copy()
counts_month = nights_property.groupby('month').agg({'Count':'sum'}).reset_index().rename(columns={'Count':'total_counts'})

In [99]:
#HTML(nights_property.to_html())
sum_supplies = sum_supplies.merge(counts_month, left_on='yearmonth', right_on='month', how='left').copy()
sum_supplies = sum_supplies.loc[sum_supplies['total_counts'].notna()].copy()
sum_supplies['supply_per_count'] = (
    sum_supplies['Amount'] / sum_supplies['total_counts'].replace(0, pd.NA)
)
(sum_supplies['Amount'].sum() / sum_supplies['total_counts'].sum())

np.float64(0.9333738313206079)